In [1]:
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2

#import opencv as cv2

img_array = np.load('test_images.npy',encoding='latin1')

images = np.load('train_images.npy',encoding='latin1')

In [5]:
import csv
def readCSV(path):
    with open(path) as csv_file:
        result = []
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            line = []
            for data in row:
                line.append(data)
            result.append(line)
    return result

In [83]:
input=[]
for i in range(len(images)):
    image1 = (images[i][1]).reshape(100,100)
    #plt.imshow(image1)
    img = cv2.imwrite('temp.jpg',image1)
    img = cv2.imread('temp.jpg',0)
    edges = cv2.Canny(img, 0, 100)
    #plt.imshow(edges)

    im2, contours, hierarchy = cv2.findContours(edges, cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(contours, key = cv2.contourArea, reverse = True)[:1]


    mask=np.zeros(img.shape)
    cv2.drawContours(mask, cnts, -1, (255),1)
    input.append(mask)
    #plt.imshow(mask)


In [84]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import ShuffleSplit
labels = np.array(readCSV('train_labels.csv'))#np.loadtxt('train_labels.csv',dtype=str,delimiter=',')
X = [x.reshape(10000,) for x in input]
y = labels[1:,1].tolist()

# print(y)

In [89]:
# train = [a + b for a, b in zip(X, y)]
train = []
for i in range(len(X)):
    a = []
    for k in X[i]:
        a.append(k)
    a.append(y[i])
    train.append(a)
# print(train[0])

In [8]:
from random import seed
from random import random

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


## forward propagation

In [13]:
import math
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + math.exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [14]:
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


## Back propagation error

In [16]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [17]:
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002711797799238243}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14813473120687762}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.05472601157879688}]


In [18]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

In [19]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [91]:
# Test training backprop algorithm
seed(1)
# dataset = [[2.7810836,2.550537003,0],
#     [1.465489372,2.362125076,0],
#     [3.396561688,4.400293529,0],
#     [1.38807019,1.850220317,0],
#     [3.06407232,3.005305973,0],
#     [7.627531214,2.759262235,1],
#     [5.332441248,2.088626775,1],
#     [6.922596716,1.77106367,1],
#     [8.675418651,-0.242068655,1],
#     [7.673756466,3.508563011,1]]
n_inputs = len(X)
n_outputs = (len(set(y)))
# print(len(X))
# print(len(set(y)))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, train, 0.3, 20, n_outputs)
for layer in network:
    print(layer)

TypeError: list indices must be integers or slices, not str